In [2]:
import warnings
warnings.filterwarnings("ignore")
import torch
import transformers
import evaluate
import numpy as np
print(torch.backends.mps.is_available())  # Should return True
print(torch.backends.mps.is_built())  # Should return True

True
True


In [3]:
print("Torch version:", torch.__version__)
print("Transformers version:", transformers.__version__)

Torch version: 2.0.1
Transformers version: 4.24.0


In [4]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from peft import get_peft_model, LoraConfig

In [5]:
# Step 1: Load a pre-trained model and tokenizer
model_checkpoint = "distilbert-base-uncased"  # You can choose any model you'd like
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [6]:
# Step 2: Load a dataset (e.g., IMDB sentiment analysis dataset)
dataset = load_dataset("imdb")
# Take smaller subsets from train and test for faster training
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(3000))
dataset["test"] = dataset["test"].shuffle(seed=42).select(range(1000))

In [7]:
dataset["test"]

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})

In [8]:
# Step 3: Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [9]:
train_dataset = dataset["train"].map(tokenize_function, batched=True)
test_dataset = dataset["test"].map(tokenize_function, batched=True)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 3786.27 examples/s]


In [10]:
test_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

In [11]:
# Step 4: Define LoRA configuration for efficient fine-tuning
lora_config = LoraConfig(
    task_type="SEQ_CLS",
    r=8,  # Low rank dimension for LoRA
    lora_alpha=32,  # Scaling factor for LoRA
    lora_dropout=0.1,  # Dropout for LoRA layers
    target_modules=['q_lin'],  # LoRA will be applied to query layers
)


In [12]:
# Step 5: Prepare the model with LoRA
# define label maps
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2
)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.we

In [13]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1257988 || all params: 67620868 || trainable%: 1.8603547058875376


In [14]:
# 6. Define Trainer args
training_args = TrainingArguments(
    output_dir="./lora-distilbert-imdb",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    num_train_epochs=2,
    learning_rate=2e-4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
    load_best_model_at_end=True,
)

In [15]:
# 7. Evaluation
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return accuracy.compute(predictions=preds, references=labels)

In [16]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# 8. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [17]:
#9. Train the model
trainer.train()

The following columns in the training set don't have a corresponding argument in `PeftModelForSequenceClassification.forward` and have been ignored: text. If text are not expected by `PeftModelForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3000
  Num Epochs = 2
  Instantaneous batch size per device = 20
  Total train batch size (w. parallel, distributed & accumulation) = 20
  Gradient Accumulation steps = 1
  Total optimization steps = 300
  Number of trainable parameters = 1257988
  3%|▎         | 10/300 [01:27<41:48,  8.65s/it]

{'loss': 0.6884, 'learning_rate': 0.00019333333333333333, 'epoch': 0.07}


  7%|▋         | 20/300 [02:54<40:37,  8.71s/it]

{'loss': 0.6634, 'learning_rate': 0.0001866666666666667, 'epoch': 0.13}


 10%|█         | 30/300 [04:21<38:58,  8.66s/it]

...

                                                   
100%|██████████| 300/300 [1:45:39<00:00,  8.56s/it]Saving model checkpoint to ./lora-distilbert-imdb/checkpoint-300
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


{'eval_loss': 0.30451661348342896, 'eval_accuracy': 0.877, 'eval_runtime': 138.2998, 'eval_samples_per_second': 7.231, 'eval_steps_per_second': 0.362, 'epoch': 2.0}


tokenizer config file saved in ./lora-distilbert-imdb/checkpoint-300/tokenizer_config.json
Special tokens file saved in ./lora-distilbert-imdb/checkpoint-300/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./lora-distilbert-imdb/checkpoint-300 (score: 0.30451661348342896).
100%|██████████| 300/300 [1:45:39<00:00, 21.13s/it]

{'train_runtime': 6339.9275, 'train_samples_per_second': 0.946, 'train_steps_per_second': 0.047, 'train_loss': 0.3517199206352234, 'epoch': 2.0}


TrainOutput(global_step=300, training_loss=0.3517199206352234, metrics={'train_runtime': 6339.9275, 'train_samples_per_second': 0.946, 'train_steps_per_second': 0.047, 'train_loss': 0.3517199206352234, 'epoch': 2.0})

In [18]:
metrics = trainer.evaluate()
print(metrics)

The following columns in the evaluation set don't have a corresponding argument in `PeftModelForSequenceClassification.forward` and have been ignored: text. If text are not expected by `PeftModelForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 20
100%|██████████| 50/50 [02:16<00:00,  2.74s/it]

{'eval_loss': 0.30451661348342896, 'eval_accuracy': 0.877, 'eval_runtime': 139.7126, 'eval_samples_per_second': 7.158, 'eval_steps_per_second': 0.358, 'epoch': 2.0}
